In [7]:
import tkinter as tk
import nltk
from textblob import TextBlob
from newspaper import Article
from os import listdir
import string
from pickle import dump, load
import pandas as pd
import numpy as np
import re

In [41]:
AMAZON_DATA_PATH= 'Reviews.csv'

In [42]:
class Load_amazon_data:
    def __init__(self, dir_path, seed= 0):
        self.dir_path= dir_path
        np.random.seed(seed)
        
    def load(self):
        return pd.read_csv(self.dir_path)
    
    def drop(self):
        data= self.load()
        
        data = data.dropna()
        data= data.iloc[:, -2:]
        data = data.reset_index(drop= True)
        
        return data
    
    def analyze_data(self):
        data= self.drop()
        
        for sr_no, i in enumerate(np.random.randint(10, 100, size= 5)):
            print("_________________________")
            print("Data Point {0}".format(sr_no + 1))
            print("Summary:")
            print(data['Summary'].iloc[i])
            print("Full Text:")
            print(data['Text'].iloc[i])

In [43]:
obj= Load_amazon_data(AMAZON_DATA_PATH, seed= 1)

In [44]:
data= obj.load()
data.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [45]:
data= obj.drop()
data.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


In [46]:
obj.analyze_data()

_________________________
Data Point 1
Summary:
Mushy
Full Text:
The flavors are good.  However, I do not see any differce between this and Oaker Oats brand - they are both mushy.
_________________________
Data Point 2
Summary:
Delicious product!
Full Text:
I can remember buying this candy as a kid and the quality hasn't dropped in all these years. Still a superb product you won't be disappointed with.
_________________________
Data Point 3
Summary:
Forget Molecular Gastronomy - this stuff rockes a coffee creamer!
Full Text:
I know the product title says Molecular Gastronomy, but don't let that scare you off.  I have been looking for this for a while now, not for food science, but for something more down to earth.  I use it to make my own coffee creamer.<br /><br />I have to have my coffee blonde and sweet - but the flavored creamers are full of the bad kinds of fat, and honestly, I hate to use manufactured "food" items.  I really don't think they are good for the body.  On the other h

In [47]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [48]:
class Data_cleaning:
    def __init__(self):
        self.clean_summaries= []
        self.clean_texts= []

    def clean_text(self, text, remove_stopwords = False):
        """
        Defines a series of cleaning operations 
        """
        text = text.lower()

        if True:
            text = text.split()
            new_text = []
            for word in text:
                if word in contractions:
                    new_text.append(contractions[word])
                else:
                    new_text.append(word)
            text = " ".join(new_text)

        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text) 
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'<br >', ' ', text)
        text = re.sub(r'<br  >', ' ', text)
        text = re.sub(r'\'', ' ', text)

        # Optionally, remove stop words
        if remove_stopwords:
            text = text.split()
            stops = set(stopwords.words("english"))
            text = [w for w in text if not w in stops]
            text = " ".join(text)

        return text
    
    def clean(self, data):
        """
        Applies the clean_text() to the entire dataset
        """
        for summary in data.Summary:
            self.clean_summaries.append(self.clean_text(summary))

        print("Summaries are complete.")

        for text in data.Text:
            self.clean_texts.append(self.clean_text(text))

        print("Texts are complete.")
        
        return self.clean_summaries, self.clean_texts

In [49]:
# import nltk
# nltk.download('stopwords')

clean_obj= Data_cleaning()
clean_summaries, clean_texts= clean_obj.clean(data)

Summaries are complete.
Texts are complete.


In [50]:
np.random.seed(1)

for sr_no, i in enumerate(np.random.randint(10, 100, size= 5)):
    print("_________________________")
    print("Data Point #{0}".format(sr_no + 1))
    print("Summary:")
    print(clean_summaries[i])
    print("Full Text:")
    print(clean_texts[i])

_________________________
Data Point #1
Summary:
mushy
Full Text:
the flavors are good  however  i do not see any differce between this and oaker oats brand   they are both mushy 
_________________________
Data Point #2
Summary:
delicious product 
Full Text:
i can remember buying this candy as a kid and the quality has not dropped in all these years  still a superb product you will not be disappointed with 
_________________________
Data Point #3
Summary:
forget molecular gastronomy   this stuff rockes a coffee creamer 
Full Text:
i know the product title says molecular gastronomy  but do not let that scare you off  i have been looking for this for a while now  not for food science  but for something more down to earth  i use it to make my own coffee creamer   i have to have my coffee blonde and sweet   but the flavored creamers are full of the bad kinds of fat  and honestly  i hate to use manufactured  food  items  i really do not think they are good for the body  on the other hand  i

-----------------------------------------

In [51]:
class LoadData:
    def __init__(self, directory):
        self.directory= directory
        
    def load_stories(self):
        """
        Load the data and store it in a list of dictionaries
        
        """
        all_stories= list()
        
        def load_doc(filename):
            """
            Return the data from a given filename
            """
            file = open(filename, encoding='utf-8')
            text = file.read()
            file.close()
            return text
        
        def split_story(doc):
            """
            Split story from summaries based on the separater -> "@highlight"
            """
            index = doc.find('@highlight')
            story, highlights = doc[:index], doc[index:].split('@highlight')
            highlights = [h.strip() for h in highlights if len(h) > 0]
            return story, highlights
        
        list_of_files= listdir(self.directory)
        for name in list_of_files:
            filename = self.directory + '/' + name
            doc = load_doc(filename)
            story, highlights= split_story(doc)
            all_stories.append({'story': story, 'highlights': highlights})
        
        return all_stories

In [53]:
DIR_PATH= "cnnstoriestokenized"
obj= LoadData(DIR_PATH)
stories= obj.load_stories()

In [54]:
len(stories)

92579

In [55]:
print(stories[10]['highlights'])
print()
print(stories[10]['story'])

['Hawaiian Airlines again lands at No. 1 in on-time performance', 'The Airline Quality Rankings Report looks at the 14 largest U.S. airlines', 'ExpressJet and American Airlines had the worst on-time performance', 'Virgin America had the best baggage handling ; Southwest had lowest complaint rate']

-LRB- CNN -RRB- -- If you travel by plane and arriving on time makes a difference , try to book on Hawaiian Airlines . In 2012 , passengers got where they needed to go without delay on the carrier more than nine times out of 10 , according to a study released on Monday .

In fact , Hawaiian got even better from 2011 , when it had a 92.8 % on-time performance . Last year , it improved to 93.4 % .

The Airline Quality Rankings Report looks at the 14 largest U.S. airlines and is based on an analysis of U.S. Department of Transportation figures . It 's co-authored by Brent Bowen , the head of the Department of Aviation Technology at Purdue University , and Dean Headley of Wichita State .

In add

In [66]:
stories[:2]

[{'story': "It 's official : U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria .\n\nObama sent a letter to the heads of the House and Senate on Saturday night , hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons .\n\nThe proposed legislation from Obama asks Congress to approve the use of military force `` to deter , disrupt , prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction . ''\n\nIt 's a step that is set to turn an international crisis into a fierce domestic political battle .\n\nThere are key questions looming over the debate : What did U.N. weapons inspectors find in Syria ? What happens if Congress votes no ? And how will the Syrian government react ?\n\nIn a televised address from the White House Rose Garden earlier Saturday , the president said he would take his case to Congress , 

In [67]:
class Clean_data:
    def __init__(self):
        pass
           
    def clean_lines(self, lines):
        cleaned = list()
        table = str.maketrans('', '', string.punctuation)
        
        for line in lines:
            index = line.find('(CNN)')
            if index >= 0:
                line = line[index + len('(CNN)'):]

            split_line = line.split()
            
            split_line = [word.lower() for word in split_line]
            split_line = [w.translate(table) for w in split_line]
            
            split_line = [word for word in split_line if word.isalpha()]
            cleaned.append(' '.join(split_line))
        cleaned = [c for c in cleaned if len(c) > 0]
        return cleaned

In [68]:
obj1= Clean_data()
cleaned_stories= list()
for example in stories:
    cleaned_stories.append({'story': obj1.clean_lines(example['story'].split('\n')), 'highlights': obj1.clean_lines(example['highlights'])})    

In [74]:
# cleaned_stories[60]
cleaned_stories[2]

{'story': ['kansas city missouri lrb cnn rrb the general services administration already under investigation for lavish spending allowed an employee to telecommute from hawaii even though he is based at the gsa s kansas city missouri office a cnn investigation has found',
  'it cost more than for the business development specialist to travel to and from the mainland united states over the past year',
  'he is among several hundred gsa virtual workers who also travel to various conferences and their home offices costing the agency millions of dollars over the past three years',
  'under the program employees work from home and may live in another state from the region in which they re actually assigned',
  'the kansas city employee who started his job in january is paid and works from his home in honolulu a gsa representative confirmed',
  'in the past year according to gsa travel records the employee has flown back to the mainland nine times for conferences and meetings four of those t